# Creating pandapipes Networks

This tutorial will introduce the user into the pandapipes datastructure and how to create networks through the pandapipes API. The following minimal example contains the most common elements that are supported by the pandapipes format. 

<img src="pics/simple_network-1.png">

The datastructure of the pandapipes framework is based on the Python library pandas. A pandapipes network consist of separate tables for each component type that is used in the network and each table holds the elements of its component type. Each row is indexed and represents a single element. The parameters are organized in columns.

By executing the follwing code cells you generate the different component tables. You can find detailed descriptions about each parameter in the pandapipes documentation under bulletpoint "Datastructure and Components".

### Empty Network

First, we import pandapipes and create an empty network:
We have to state the name of the fluid for the network. In pandapipes, some fluids are already
pre-defined, e.g. <span style="color:green">hgas</span>, <span style="color:green">lgas</span>
(high and low calorific natural gas),  <span style="color:green">hydrogen</span>,
<span style="color:green">water</span>, <span style="color:green">biomethane_pure</span>, <span style="color:green">biomethane_treated</span> (see [here](https://pandapipes.readthedocs.io/en/latest/fluid_properties/fluids.html) for the composition) and <span style="color:green">air</span>. It is
also possible to create a fluid if needed (tutorial in preparation).

In this example, we will create a medium pressure gas network and choose <span style="color:green">lgas</span> from the predefined fluids in pandapipes.

In [1]:
import pandapipes as pp

In [2]:
net = pp.create_empty_network(fluid="lgas") # create an empty network

This network does not hold any elements yet. It serves as a container for the component tables and network specific parameters.

In [3]:
net

This pandapipes network includes the following parameter tables:.
It contains the following fluid: 
Fluid lgas (gas) with properties:
   - density (InterExtra)
   - viscosity (InterExtra)
   - heat_capacity (InterExtra)
   - molar_mass (Constant)
   - der_compressibility (Constant)
   - compressibility (Linear)
   - lhv (Constant)
   - hhv (Constant)
and uses the following component models:
   - Junction
   - Pipe
   - ExtGrid

We will now use the <span style="color:blue">create</span> functions to create elements. They will be automatically added to the respective tables. 

### Junctions

<img src="pics/simple_network-junc.png">

We now create 6 junctions with an intital pressure for the pipeflow calculation of 1.0 bar. For improved readability, we add names (optional).

In [4]:
junction1 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Connection to External Grid")
junction2 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 2")
junction3 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 3")
junction4 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 4")
junction5 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 5")
junction6 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 6")

The junctions are now in the net and can be called. Some parameters were filled with default values.

In [5]:
net.junction # show junction table

,name,pn_bar,tfluid_k,height_m,in_service,type
0,Connection to External Grid,1.0,293.15,0.0,True,junction
1,Junction 2,1.0,293.15,0.0,True,junction
2,Junction 3,1.0,293.15,0.0,True,junction
3,Junction 4,1.0,293.15,0.0,True,junction
4,Junction 5,1.0,293.15,0.0,True,junction
5,Junction 6,1.0,293.15,0.0,True,junction


All create functions return the pandapipes index of the element that was created, for example the variable junction1 is now equal to the index of the junction with the name "Connection to External Grid" (which is 0):


In [6]:
junction1

np.int64(0)

In [7]:
net.junction.loc[junction1]

name          Connection to External Grid
pn_bar                                1.0
tfluid_k                           293.15
height_m                              0.0
in_service                           True
type                             junction
Name: 0, dtype: object

We use these variables for creating bus and branch elements in the following.

### External Grid

<img src="pics/simple_network-ext_grid.png">

We now create a medium pressure external grid connection that serves as slack node for the pipe flow calculation. The pressure is set to 1.1 bar:

In [8]:
medium_pressure_grid = pp.create_ext_grid(net, junction=junction1, p_bar=1.1, t_k=293.15, name="Grid Connection")

net.ext_grid # show external grid table

,name,junction,p_bar,t_k,in_service,type
0,Grid Connection,0,1.1,293.15,True,pt


### Pipes

The network includes 5 pipes between two junctions each. The junctions and pipes lengths are defined in the network diagram:

<img src="pics/simple_network-pipes.png">

For all pipes we want a diameter of 300 mm. 

In [9]:
pipe1 = pp.create_pipe_from_parameters(net, from_junction=junction1, to_junction=junction2, length_km=10, diameter_m=0.3, name="Pipe 1")
pipe2 = pp.create_pipe_from_parameters(net, from_junction=junction2, to_junction=junction3, length_km=2, diameter_m=0.3, name="Pipe 2")
pipe3 = pp.create_pipe_from_parameters(net, from_junction=junction2, to_junction=junction4, length_km=2.5, diameter_m=0.3, name="Pipe 3")
pipe4 = pp.create_pipe_from_parameters(net, from_junction=junction3, to_junction=junction5, length_km=1, diameter_m=0.3, name="Pipe 4")
pipe5 = pp.create_pipe_from_parameters(net, from_junction=junction4, to_junction=junction6, length_km=1, diameter_m=0.3, name="Pipe 5")

The full pipe table looks like this:

In [10]:
net.pipe # show pipe table

,name,from_junction,to_junction,std_type,length_km,diameter_m,k_mm,loss_coefficient,u_w_per_m2k,text_k,qext_w,sections,in_service,type
0,Pipe 1,0,1,None,10.0,0.3,0.2,0.0,0.0,NaN,0.0,1,True,pipe
1,Pipe 2,1,2,None,2.0,0.3,0.2,0.0,0.0,NaN,0.0,1,True,pipe
2,Pipe 3,1,3,None,2.5,0.3,0.2,0.0,0.0,NaN,0.0,1,True,pipe
3,Pipe 4,2,4,None,1.0,0.3,0.2,0.0,0.0,NaN,0.0,1,True,pipe
4,Pipe 5,3,5,None,1.0,0.3,0.2,0.0,0.0,NaN,0.0,1,True,pipe


The parameters from_junction and to_junction define the orientation of the pipe. If the fluid streams in fact from the from_junction to to_junction, the resulting fluid velocity is positive. If the fluid stream direction is opposite, the resulting velocity is negative.

### Valve

There is one valve between Junction 5 and Juncion 6. It is open so that the gas can circulate.

<img src="pics/simple_network-valve.png">

In [11]:
valve = pp.create_valve(net, junction=junction5, element=junction6, et='ju', diameter_m=0.3, opened=True)

As we see in the valve table below the default value for the loss_coefficient of the valve is zero. Also, all valves have a length of 0 meter, because we consider ideal valves. However, if needed you can change the loss_coefficient parameter. 

In [12]:
net.valve # show valve table

,name,junction,element,et,diameter_m,opened,loss_coefficient,type
0,None,4,5,ju,0.3,True,0.0,valve


### Sink

<img src="pics/simple_network-sink.png">

The sink element is used to model by default a constant consumption. We create here a gas consumption sink with a mass flow of 545 gramm per second. Sinks and sources always have to be assigned to a particular junction. The sign of the mass flow is positive.

In [13]:
sink = pp.create_sink(net, junction=junction4, mdot_kg_per_s=0.545, name="Sink 1")
net.sink

,name,junction,mdot_kg_per_s,scaling,in_service,type
0,Sink 1,3,0.545,1.0,True,sink


### Source

<img src="pics/simple_network-source.png">

The source element is used to model a generation of heat or gas. It could be for example a biogas plant that is directly connected to the gas network or a combined heat and power plant.
In this example, we assume that a biogas plant is directly feeding into the gas grid with a constant value of 234 gramm per second. The sign of the mass flow is also positive.

In [14]:
source = pp.create_source(net, junction=junction3, mdot_kg_per_s=0.234, name="Source 1")

In [15]:
net.source

,name,junction,mdot_kg_per_s,scaling,in_service,type
0,Source 1,2,0.234,1.0,True,source


We are done. Now, the net includes all the elements from the picture.

In [16]:
net

This pandapipes network includes the following parameter tables:
   - junction (6 elements)
   - pipe (5 elements)
   - ext_grid (1 elements)
   - valve (1 elements)
   - sink (1 elements)
   - source (1 elements).
It contains the following fluid: 
Fluid lgas (gas) with properties:
   - density (InterExtra)
   - viscosity (InterExtra)
   - heat_capacity (InterExtra)
   - molar_mass (Constant)
   - der_compressibility (Constant)
   - compressibility (Linear)
   - lhv (Constant)
   - hhv (Constant)
and uses the following component models:
   - Junction
   - Pipe
   - ExtGrid
   - Valve
   - Sink
   - Source

Finally, the gas flow can be calculated with the `pipeflow` command:

In [17]:
pp.pipeflow(net)

The results are written to `res_...` tables. They were added to the `net`container. 

In [18]:
net # result tables have been added to the net 

This pandapipes network includes the following parameter tables:
   - junction (6 elements)
   - pipe (5 elements)
   - ext_grid (1 elements)
   - valve (1 elements)
   - sink (1 elements)
   - source (1 elements)
and the following results tables:
   - res_junction (6 elements)
   - res_pipe (5 elements)
   - res_ext_grid (1 elements)
   - res_valve (1 elements)
   - res_sink (1 elements)
   - res_source (1 elements).
It contains the following fluid: 
Fluid lgas (gas) with properties:
   - density (InterExtra)
   - viscosity (InterExtra)
   - heat_capacity (InterExtra)
   - molar_mass (Constant)
   - der_compressibility (Constant)
   - compressibility (Linear)
   - lhv (Constant)
   - hhv (Constant)
and uses the following component models:
   - Junction
   - Pipe
   - ExtGrid
   - Valve
   - Sink
   - Source

In [19]:
net.res_junction # calculated pressure and temperature at junctions

,p_bar,t_k
0,1.100000,293.15
1,1.061641,293.15
2,1.061396,293.15
3,1.054845,293.15
4,1.058123,293.15
5,1.058123,293.15


In [20]:
net.res_pipe  # velocities, mass flows through pipes and other results



,v_from_m_per_s,v_to_m_per_s,v_mean_m_per_s,p_from_bar,p_to_bar,t_from_k,t_to_k,t_outlet_k,mdot_from_kg_per_s,mdot_to_kg_per_s,vdot_norm_m3_per_s,reynolds,lambda,normfactor_from,normfactor_to
0,2.818661,2.871013,2.844518,1.100000,1.061641,293.15,293.15,293.15,0.311000,-0.311000,0.388996,111395.955082,0.018389,0.512189,0.521703
1,0.474650,0.474706,0.474678,1.061641,1.061396,293.15,293.15,293.15,0.051416,-0.051416,0.064311,18416.653950,0.021290,0.521703,0.521764
2,2.396364,2.404274,2.400310,1.061641,1.054845,293.15,293.15,293.15,0.259584,-0.259584,0.324685,92979.301132,0.018503,0.521703,0.523425
3,2.635146,2.639329,2.637235,1.061396,1.058123,293.15,293.15,293.15,0.285416,-0.285416,0.356995,102232.260024,0.018441,0.521764,0.522593
4,-2.643532,-2.639329,-2.641428,1.054845,1.058123,293.15,293.15,293.15,-0.285416,0.285416,-0.356995,102232.260024,0.018441,0.523425,0.522593
